# MIIIT - The Metabolomics Interactive Intensity Integration Tool

In [1]:
import os
#os.chdir('results')

In [2]:
from ipywidgets import Button
import uuid
import datetime
import ipywidgets as widgets
import traitlets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
from tkinter import Tk, filedialog
from tqdm import tqdm_notebook
from pyteomics import mzxml
import pandas as pd

class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        try:
            # Create Tk root
            root = Tk()
            # Hide the main window
            root.withdraw()
            # Raise the root to the top of all windows.
            root.call('wm', 'attributes', '.', '-topmost', True)
            # List of selected fileswill be set to b.value
            b.files = filedialog.askopenfilename(multiple=True)
        except:
            pass

In [3]:
def integrate_peak(filename, rt, delta_rt, mz, delta_mz, peaklabel=None):
    if peaklabel is None:
        peaklabel = 'Intensity'
    df = mzxml_to_pandas_df(filename)
    intensity = slice_ms1_mzxml(df, rt, delta_rt, mz, delta_mz)\
                    ['intensity array'].sum().sum()
    result = pd.DataFrame({'RT': [rt], 
                           'DetaRT': [delta_rt],
                           'MZ': [mz],
                           'DeltaMZ': [delta_mz],
                           peaklabel: [intensity]}, index=[filename])
    result.index.name = 'FileName'
    return result

def mzxml_to_pandas_df(filename):
    slices = []
    file = mzxml.MzXML(filename)
    while True:
        try:
            slices.append(pd.DataFrame(file.next()))
        except:
            break
    df = pd.concat(slices)
    df_to_numeric(df)
    return df


def df_to_numeric(df):
    for col in df:
        df.loc[:, col] = pd.to_numeric(df[col], errors='ignore')


def slice_ms1_mzxml(df, rt, delta_rt, mz, delta_mz):
    df_slice = df.loc[(rt-delta_rt <= df.retentionTime) &
                      (df.retentionTime <= rt+delta_rt) &
                      (mz-delta_mz <= df['m/z array']) & 
                      (df['m/z array'] <= mz+delta_mz)]
    return df_slice

In [6]:
from IPython.display import clear_output
from ipywidgets import Button, HBox, VBox, Textarea, HTML
import time

class App():
    def __init__(self, mzxmlfiles):
        self.files = mzxmlfiles
        self.rt = widgets.BoundedFloatText(
            value=2,
            min=0,
            max=100000.0,
            step=0.001,
            description='RT:',
            disabled=False)
        self.drt = widgets.BoundedFloatText(
            value=1,
            min=0,
            max=15,
            step=0.01,
            description='Delta RT:',
            disabled=False)
        self.mz = widgets.BoundedFloatText(
            value=500,
            min=0,
            max=100000.0,
            step=0.001,
            description='m/z:',
            disabled=False)
        self.dmz = widgets.BoundedFloatText(
            value=0.1,
            min=0,
            max=100000.0,
            step=0.001,
            description='Delta m/z:',
            disabled=False)
        self.message_box = Textarea(
            value='',
            placeholder='Please select some files and click on Run.',
            description='',
            disabled=True,
            layout={'width': '95%', 'height': '500px', 'font_family': 'monospace'})
        self.list_button = Button(description="List Files")
        self.list_button.on_click(self.list_files)
        self.run_button = Button(description="Run")
        self.run_button.on_click(self.run)
        self.download_button = Button(description="Download")
        self.download_button.on_click(self.download)
        self.results = None
        self.download_html = HTML("""Nothing to download""")
        
    def run(self, b):
        # print('Running')
        rt = self.rt.value
        drt = self.drt.value 
        mz = self.mz.value
        dmz = self.dmz.value
        # print('Parameters: rt={}+/-{}, mz={}+/-{}'.format(rt, drt, mz, dmz))
        time.sleep(1)
        results = []
        for filename in tqdm_notebook(self.files.files):
            #print('Processing:', filename)
            result = integrate_peak(filename, rt, drt, mz, dmz, 'IntensitySum')
            results.append(result)
        self.results = pd.concat(results)
        self.message_box.value = self.results.to_string()
        self.download(None)
        return self.results

    def list_files(self, b):
        text = ''
        for line in self.files.files:
            text += line+'\n'
        self.message_box.value = text
    
    def download(self, b):
        if self.results is None:
            print('First you have to create some results.')
        else:
            uid = str(uuid.uuid4()).split('-')[-1]
            now = datetime.datetime.now().strftime("%Y-%m-%d")
            filename = '{}-metabolomics_peak_intensity-{}.csv'.format(now, uid)
            self.results.to_csv(filename)
            self.download_html.value = """<a download='{}' href='{}'>Download</a>""".format(filename, filename)

mzxmlfiles = SelectFilesButton()
app = App(mzxmlfiles)

VBox([HBox([mzxmlfiles, app.list_button, app.run_button, app.download_html]),
      HBox([app.rt, app.drt, app.mz, app.dmz]),
      app.message_box])
